# Bulk Delete Analyses (From Config File)

This notebook deletes Analyses from Moody's Risk Modeler based on analysis names defined in an Excel configuration file.

**Use Case:** Clean up analyses that were created from a specific configuration file (e.g., test data cleanup).

**Process:**
1. Select an Excel configuration file from the `configuration/` folder
2. Read analysis names from the "Analysis Table" tab
3. Search for these analyses in Moody's using Database (EDM) + Analysis Name
4. Review found analyses and any not found
5. Confirm deletion
6. Delete analyses with progress tracking
7. View summary of results

**Warning:** This action permanently deletes analyses from Moody's and CANNOT be undone!

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from pathlib import Path
from datetime import datetime
from helpers import ux
from helpers.irp_integration import IRPClient
from helpers.irp_integration.exceptions import IRPAPIError

# Initialize IRP client
irp_client = IRPClient()

ux.header("Bulk Delete Analyses (From Config File)")
ux.info("This tool deletes Analyses based on names in an Excel configuration file.")

## 1. Select Configuration File

Place your Excel configuration file in the `configuration/` folder, then select it below.

In [ ]:
ux.subheader("Select Configuration File")

# Get the configuration directory (relative to this notebook)
config_dir = Path("configuration")
ux.info(f"Configuration directory: {config_dir.absolute()}")
print()

# Check if directory exists
if not config_dir.exists():
    ux.warning(f"Configuration directory does not exist: {config_dir}")
    ux.info("Creating directory...")
    config_dir.mkdir(parents=True, exist_ok=True)
    ux.success("Directory created")

# List available Excel files in the directory
excel_files = sorted(list(config_dir.glob("*.xlsx")) + list(config_dir.glob("*.xls")))

if not excel_files:
    ux.error("No Excel files found in configuration directory")
    print()
    ux.info(f"Please place your configuration file in: {config_dir.absolute()}")
    ux.info("The file should contain an 'Analysis Table' sheet with 'Database' and 'Analysis Name' columns.")
    raise FileNotFoundError("No configuration files found in directory")

# Display available files
ux.info(f"Found {len(excel_files)} Excel file(s):")
print()
for i, f in enumerate(excel_files, 1):
    file_size = f.stat().st_size / 1024  # KB
    file_modified = datetime.fromtimestamp(f.stat().st_mtime).strftime('%Y-%m-%d %H:%M')
    ux.info(f"  [{i}] {f.name}")
    ux.info(f"      Size: {file_size:.2f} KB | Modified: {file_modified}")
    print()

# Get user selection
selection = ux.text_input(
    f"Select configuration file [1-{len(excel_files)}]",
    default="1"
)

if selection is None:
    ux.warning("Operation cancelled by user")
    raise Exception("User cancelled operation")

try:
    selected_index = int(selection) - 1
    if selected_index < 0 or selected_index >= len(excel_files):
        raise ValueError("Selection out of range")
    config_path = excel_files[selected_index]
except (ValueError, IndexError) as e:
    ux.error(f"Invalid selection: {selection}")
    raise ValueError(f"Invalid file selection. Please enter a number between 1 and {len(excel_files)}")

# Display selected file information
ux.success(f"Selected: {config_path.name}")
file_size = config_path.stat().st_size / 1024
file_modified = config_path.stat().st_mtime

file_info = [
    ["File Name", config_path.name],
    ["File Size", f"{file_size:.2f} KB"],
    ["Last Modified", datetime.fromtimestamp(file_modified).strftime('%Y-%m-%d %H:%M:%S')]
]
ux.table(file_info, headers=["Property", "Value"])

## 2. Read Analysis Names from Configuration

Reading the "Analysis Table" tab and extracting Database (EDM) and Analysis Name columns.

In [ ]:
ux.subheader("Reading Configuration File")

try:
    # Read the Analysis Table sheet
    df = pd.read_excel(config_path, sheet_name='Analysis Table')
    ux.success(f"Successfully read 'Analysis Table' sheet")
    
except ValueError as e:
    if "Analysis Table" in str(e):
        ux.error("'Analysis Table' sheet not found in the configuration file")
        ux.info("The Excel file must contain a sheet named 'Analysis Table'")
    raise
except Exception as e:
    ux.error(f"Failed to read configuration file: {e}")
    raise

# Check for required columns
required_columns = ['Database', 'Analysis Name']
missing_columns = [col for col in required_columns if col not in df.columns]

if missing_columns:
    ux.error(f"Missing required column(s): {', '.join(missing_columns)}")
    ux.info(f"Available columns: {', '.join(df.columns)}")
    raise ValueError(f"Missing required columns: {missing_columns}")

# Extract unique EDM + Analysis Name combinations
# Drop rows where either column is NaN
analysis_df = df[['Database', 'Analysis Name']].dropna()
analysis_df = analysis_df.drop_duplicates()

if analysis_df.empty:
    ux.warning("No analysis entries found in the 'Analysis Table' sheet")
    raise ValueError("No analyses to process")

# Build list of (edm_name, analysis_name) tuples
analysis_entries = list(analysis_df.itertuples(index=False, name=None))

ux.success(f"Found {len(analysis_entries)} unique analysis entry(ies) in configuration")
print()

# Display analysis entries
ux.subheader("Analyses from Configuration")
analysis_list_data = [[i, edm, name] for i, (edm, name) in enumerate(analysis_entries, 1)]
ux.table(analysis_list_data, headers=["#", "Database (EDM)", "Analysis Name"])

## 3. Search for Analyses in Moody's

Searching for each analysis by Database (EDM) + Analysis Name combination.

In [ ]:
ux.subheader("Searching Moody's API")
ux.info(f"Searching for {len(analysis_entries)} analysis(es)...")
print()

found_analyses = []
not_found_analyses = []

for i, (edm_name, analysis_name) in enumerate(analysis_entries, 1):
    try:
        # Search for exact EDM + analysis name match
        # Note: 'exposureName' is the API field for EDM name
        filter_str = f'analysisName = "{analysis_name}" AND exposureName = "{edm_name}"'
        results = irp_client.analysis.search_analyses(filter=filter_str)
        
        if results:
            found_analyses.extend(results)
            ux.success(f"  [{i}/{len(analysis_entries)}] Found: {edm_name} / {analysis_name}")
        else:
            not_found_analyses.append((edm_name, analysis_name))
            ux.warning(f"  [{i}/{len(analysis_entries)}] Not found: {edm_name} / {analysis_name}")
            
    except IRPAPIError as e:
        not_found_analyses.append((edm_name, analysis_name))
        ux.error(f"  [{i}/{len(analysis_entries)}] Error searching for: {edm_name} / {analysis_name}")
        ux.warning(f"      {str(e)[:80]}")

print()
ux.info(f"Search complete: {len(found_analyses)} found, {len(not_found_analyses)} not found")

## 4. Review Analyses to Delete

Review the analyses that were found and will be deleted.

In [ ]:
# Show analyses not found (if any)
if not_found_analyses:
    ux.subheader(f"Analyses Not Found ({len(not_found_analyses)})")
    ux.warning("The following analyses were not found in Moody's (already deleted or never created):")
    for edm, name in not_found_analyses:
        ux.info(f"  - {edm} / {name}")
    print()

# Show found analyses
if not found_analyses:
    ux.warning("No analyses found in Moody's to delete")
    ux.info("All analyses from the configuration are either already deleted or were never created.")
    raise Exception("No analyses to delete")

ux.subheader(f"Analyses Found ({len(found_analyses)})")

# Build DataFrame for display
analyses_data = []
for a in found_analyses:
    analyses_data.append({
        'Analysis ID': a.get('analysisId'),
        'App Analysis ID': a.get('appAnalysisId'),
        'Analysis Name': a.get('analysisName'),
        'EDM': a.get('exposureName', 'N/A'),
        'Peril': a.get('perilCode', 'N/A'),
        'Region': a.get('regionCode', 'N/A'),
        'Framework': a.get('analysisFramework', 'N/A'),
        'Run Date': a.get('createDate', 'N/A')
    })

analyses_df = pd.DataFrame(analyses_data)
ux.dataframe(analyses_df, title=f"Analyses to Delete ({len(analyses_df)} total)")

# Store for deletion step
analyses_to_delete = found_analyses.copy()

print()
ux.warning(f"The above {len(analyses_to_delete)} analysis(es) will be PERMANENTLY DELETED from Moody's.")

## 5. Confirm Deletion

**This action cannot be undone!** Please review the analyses listed above carefully before confirming.

In [ ]:
ux.header("CONFIRMATION REQUIRED")
ux.error(f"You are about to PERMANENTLY DELETE {len(analyses_to_delete)} analysis(es)")
ux.info(f"Configuration file: {config_path.name}")
print()

# First confirmation
if not ux.yes_no("Are you sure you want to delete these analyses?"):
    ux.info("Operation cancelled by user")
    raise Exception("User cancelled operation")

# Second confirmation with count
if not ux.yes_no(f"Final confirmation: DELETE {len(analyses_to_delete)} analysis(es)?"):
    ux.info("Operation cancelled by user")
    raise Exception("User cancelled operation")

ux.success("Confirmation received. Starting deletion...")
print()

# Track results
deleted_count = 0
failed_count = 0
errors = []

ux.subheader("Deletion Progress")

for i, analysis in enumerate(analyses_to_delete, 1):
    analysis_name = analysis.get('analysisName', 'Unknown')
    edm_name = analysis.get('exposureName', 'Unknown')
    analysis_id = analysis.get('analysisId')
    
    try:
        # Delete the analysis
        irp_client.analysis.delete_analysis(analysis_id)
        deleted_count += 1
        ux.success(f"  [{i}/{len(analyses_to_delete)}] Deleted: {edm_name} / {analysis_name}")
        
    except Exception as e:
        failed_count += 1
        error_msg = f"{edm_name} / {analysis_name}: {str(e)}"
        errors.append(error_msg)
        ux.error(f"  [{i}/{len(analyses_to_delete)}] FAILED: {edm_name} / {analysis_name}")
        ux.warning(f"      Error: {str(e)[:100]}")

print()

## 6. Deletion Summary

In [ ]:
ux.header("Deletion Summary")

# Summary statistics
summary_data = [
    ["Configuration File", config_path.name],
    ["Analyses in Config", len(analysis_entries)],
    ["Analyses Found in Moody's", len(found_analyses)],
    ["Analyses Not Found", len(not_found_analyses)],
    ["Successfully Deleted", deleted_count],
    ["Failed to Delete", failed_count],
]

ux.table(summary_data, headers=["Metric", "Value"])
print()

# Overall status
if failed_count == 0 and deleted_count > 0:
    ux.success(f"All {deleted_count} analysis(es) deleted successfully!")
elif deleted_count == 0 and failed_count > 0:
    ux.error(f"All {failed_count} deletion(s) failed")
elif deleted_count > 0 and failed_count > 0:
    ux.warning(f"Partial success: {deleted_count} deleted, {failed_count} failed")

# Show analyses not found
if not_found_analyses:
    print()
    ux.subheader("Analyses Not Found (Skipped)")
    for edm, name in not_found_analyses:
        ux.info(f"  - {edm} / {name}")

# Show errors if any
if errors:
    print()
    ux.subheader("Deletion Errors")
    for error in errors:
        ux.error(f"  - {error}")

print()
ux.info("Note: Deleted analyses cannot be recovered. If you need to recreate them,")
ux.info("you will need to re-run the analysis workflow with the configuration file.")

---